## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 scikit-learn 建立鳶尾花（Iris）邏輯迴歸分類器並將其導出為 ONNX 格式的範例。

### 1.1 建立並訓練邏輯迴歸模型

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 載入鳶尾花資料集
iris = load_iris()
X, y = iris.data, iris.target

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# 建立模型Pipeline：標準化 + 邏輯迴歸
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 訓練模型
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression(max_iter=1000))])

### 1.2 將模型轉換為 ONNX 格式
使用 skl2onnx 將訓練好的 scikit-learn 模型轉換為 ONNX 格式：

In [3]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# 定義輸入類型
initial_type = [('float_input', FloatTensorType([None, X.shape[1]]))]

# 轉換為 ONNX 模型
onnx_model = convert_sklearn(model, initial_types=initial_type, target_opset=9)

# 指定儲存路徑
onnx_file_path = "logistic_model.onnx"

# 將 ONNX 模型儲存為檔案
with open(onnx_file_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"ONNX 模型已儲存至 {onnx_file_path}")

ONNX 模型已儲存至 ../tf-example/logistic_model.onnx


#### 1.2.1 使用ONNX Runtime進行推論測試
我們可以先透過 ONNX Runtime 輸入一筆測試資料檢查推論結果。可以跟稍後 ONNX-MLIR 推論結果進行驗證比較看有沒有數值一至。

In [3]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('logistic_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6. , 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})[1]

# 輸出預測結果
print(pred_onnx)

[{0: 2.4887262043193914e-05, 1: 0.008561260998249054, 2: 0.9914138317108154}]


### 1.3 使用 Hummingbird 將模型轉換為 ONNX 格式
TVM 的設計主要針對深度學習（DL）模型，例如 CNN、RNN 等神經網路，這些模型的特點是以張量（Tensor）為主要數據結構。 Logistic Regression 模型轉換為 ONNX 時，輸出的預測結果默認是 `Sequence<Map>` 類型，用於將分類概率與標籤對應起來。然而，這些類型並不是 TVM 所支持的張量類型，因此引發錯誤。

> OpNotImplemented: The following operators are not supported for frontend ONNX: ZipMap, Scaler, LinearClassifier, Normalizer

一個有效的解決方案是使用 Hummingbird，這是一個專門將傳統機器學習模型轉換為神經網路框架（如 PyTorch）的工具。 Hummingbird 可以將 Logistic Regression、Random Forest 等 scikit-learn 模型轉換為 PyTorch 模型。 轉換後的模型具有純張量輸入輸出結構，非常適合使用 TVM 編譯。

In [4]:
# ! pip install onnxruntime==1.19.2 onnx==1.16.1 hummingbird-ml

In [8]:
from hummingbird.ml import convert
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load('logistic_model.onnx')
# 將模型轉換為 Hummingbird 格式
hb_model = convert(onnx_model, 'onnx')
# 保存轉換後的 ONNX 模型
hb_model.save('onnx-tmp')

# 解壓縮資料夾
!unzip -o onnx-tmp.zip -d dist

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Model saved with digest: f96b10de0b3cf9f3886187ec9e352468abd4b7cf
Archive:  onnx-tmp.zip
  inflating: dist/model_type.txt     
  inflating: dist/container.pkl      
  inflating: dist/model_configuration.txt  
  inflating: dist/deploy_model.onnx  


### 1.3.1 ONNX Runtime 執行推論
使用 ONNX Runtime 執行推論 hummingbird 轉換後的模型

In [11]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('./dist/deploy_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6. , 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})[1]

# 輸出預測結果
print(pred_onnx)

[[2.4887262e-05 8.5612610e-03 9.9141383e-01]]


## 2. 使用 TVM 轉換模型為共享庫
### 2.1 將 ONNX 模型編譯為共享庫
使用 TVM relay 將 tf_model.onnx 模型轉換為共享庫（.so 文件）。

- 載入模型：從 ONNX 格式載入預訓練的模型。
- 模型轉換：將 ONNX 模型轉換為 TVM 的 Relay 模組，方便進行優化和編譯。
- 圖優化：應用高級優化策略，提高模型效率。
- 中間表示轉換：將優化後的 Relay 模組轉換為 LLVM 的中間表示。
- 代碼生成：從 LLVM IR 生成位元碼，編譯為機器代碼。
- 連結與生成庫：將機器代碼連結成最終的共享庫，供部署使用。

> 成功輸出後即可前往第3撰寫 C++ 程式進行推論

In [3]:
%%time
import tvm
from tvm import relay
from tvm.contrib import cc, utils
import onnx
import sys

original_platform = sys.platform
sys.platform = "linux"
# 載入 ONNX 模型
onnx_model = onnx.load("./dist/deploy_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱可在 ONNX 模型中確認
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構，這裡假設為通用的 CPU
# target = 'llvm'
target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=aarch64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-apple-darwin")
with tvm.transform.PassContext(opt_level=1):
    lib = relay.build(mod, target, params=params)

# 編譯輸出為 C 代碼
lib.export_library("output.so", cc="x86_64-linux-gnu-gcc")
# lib.export_library("output.so", cc="aarch64-linux-gnu-gcc")
# lib.export_library("output.so", cc="clang")

# 恢復原始平台
sys.platform = original_platform

[14:11:06] /home/jovyan/project/ONNX-MLIR/tvm/src/te/schedule/bound.cc:119: not in feed graph consumer = compute(p0_red_temp, body=[T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis=[T.iter_var(k1, T.Range(0, 3), "CommReduce", "")], condition=T.bool(True), value_index=0), T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis

CPU times: user 2.03 s, sys: 1.48 s, total: 3.51 s
Wall time: 834 ms


In [1]:
!du -h output.so

108K	output.so


如果要在 macOS 跨平台編譯。在 macOS 上使用 TVM 的export_library函數時，預設會包含-undefined dynamic_lookup連結器標誌。會導致連結器報錯：
> Command line: x86_64-linux-gnu-gcc -shared -fPIC -undefined dynamic_lookup -o output.so 

遇到的錯誤是由於 macOS 特有的連結器標誌 -undefineddynamic_lookup 被傳遞給了 Linux 交叉編譯器，導致編譯失敗。為了解決這個問題，可以暫時修改sys.platform，讓 TVM 認為正在 Linux 上執行：

```py
import sys

original_platform = sys.platform
sys.platform = "linux"
```

### 2.2 使用 TVM Python API 推論(optional)
使用 TVM runtime 載入共享庫並設置輸入數據，即可執行推論。

In [5]:
from tvm.contrib import graph_executor
import numpy as np

# 在目標設備上載入共享庫
loaded_lib = tvm.runtime.load_module("output.so")
module = graph_executor.GraphModule(loaded_lib["default"](tvm.cpu()))

# 準備輸入資料
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)
# # 設定輸入數據並執行推論
module.set_input("float_input", tvm.nd.array(input_data))
module.run()
print(module.get_output(0).asnumpy())
print(module.get_output(1).asnumpy())

[2]
[[2.4887262e-05 8.5612610e-03 9.9141383e-01]]


### 2.3 使用TVM推論(optional)
若無法順利產so 因此直接使用TVM推論

In [33]:
import onnx
import tvm
from tvm import relay

# 載入 ONNX 模型
onnx_model = onnx.load("./dist/deploy_model.onnx")

# 將 ONNX 模型轉換為 TVM 的 Relay 模型格式
input_name = 'float_input'  # 確認模型的輸入名稱，可從 ONNX 模型中查看
shape_dict = {input_name: (1, 4)}  # 定義輸入形狀 (1, 4) 代表輸入的形狀
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設定編譯目標為 CPU 上的 LLVM (可以根據需求設為 "cuda" 或 "opencl" 等其他架構)
target = "llvm"
with tvm.transform.PassContext(opt_level=3):
    # 編譯 Relay 模型，將其轉換為可以運行的格式
    lib = relay.build(mod, target=target, params=params)

[14:43:23] /home/jovyan/project/ONNX-MLIR/tvm/src/te/schedule/bound.cc:119: not in feed graph consumer = compute(p0_red_temp, body=[T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis=[T.iter_var(k1, T.Range(0, 3), "CommReduce", "")], condition=T.bool(True), value_index=0), T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis

In [34]:
import numpy as np

# 使用 TVM 的 Graph Executor 模組來載入編譯後的模型
module = graph_executor.GraphModule(lib["default"](tvm.cpu(0)))

# 定義輸入資料並設定模型輸入，這裡使用一筆測試資料
input_data = np.array([[6.3, 3.3, 6. , 2.5]], dtype=np.float32)
module.set_input(input_name, input_data)

# 執行模型推理
module.run()

# 定義輸出形狀，這裡假設模型的輸出形狀為 (1, 1)
output_shape = (1, 1)  # 可根據實際模型調整
tvm_output = module.get_output(1).asnumpy()  # 獲取輸出並轉換為 NumPy 陣列

# 輸出推理結果
print(tvm_output)

[[2.4887262e-05 8.5612610e-03 9.9141383e-01]]


In [35]:
# 打印輸出形狀
tvm_output = module.get_output(0).asnumpy()
print("Output shape:", tvm_output.shape)

Output shape: (1,)


## 3. 撰寫 C++ 程式進行推論

### 3.1 撰寫 C++ 程式

> 請參考 sklearn-example/sk_inference.cpp

### 3.2 編譯程式

In [2]:
!g++ -std=c++17 -o main sk_inference.cpp \
    -I../../tvm/include \
    -I../..//tvm/3rdparty/dlpack/include \
    -I../..//tvm/3rdparty/dmlc-core/include \
    -ltvm_runtime -ldl -pthread \
    -Wno-macro-redefined

In file included from ../../tvm/include/tvm/runtime/container/base.h:28,
                 from ../../tvm/include/tvm/runtime/container/string.h:29,
                 from ../../tvm/include/tvm/runtime/module.h:31,
                 from sk_inference.cpp:2:
../../tvm/include/tvm/runtime/logging.h:594: warning: "LOG" redefined
  594 | #define LOG(level) LOG_##level
      | 
In file included from ../..//tvm/3rdparty/dmlc-core/include/dmlc/io.h:15,
                 from ../../tvm/include/tvm/runtime/module.h:29,
                 from sk_inference.cpp:2:
../..//tvm/3rdparty/dmlc-core/include/dmlc/./logging.h:263: note: this is the location of the previous definition
  263 | #define LOG(severity) LOG_##severity.stream()
      | 
In file included from ../../tvm/include/tvm/runtime/container/base.h:28,
                 from ../../tvm/include/tvm/runtime/container/string.h:29,
                 from ../../tvm/include/tvm/runtime/module.h:31,
                 from sk_inference.cpp:2:
../../tvm/incl

In [3]:
!./main

Prediction Label: 2
Prediction Probabilities: [2.48873e-05, 0.00856126, 0.991414]


In [4]:
!python -c "import tvm; print(tvm.__file__)"

[14:36:37] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[14:36:37] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[14:36:37] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
/home/jovyan/project/ONNX-MLIR/tvm/python/tvm/__init__.py


## 4. TVM其他功能

In [ ]:
!pip install apache-tvm

In [36]:
!tvmc compile ./dist/deploy_model.onnx --target="llvm" --output model.tar 

/bin/bash: line 1: tvmc: command not found


### 4.1 Micro TVM 
產生三個檔案，但不知道怎模用。
- lib1.c
- devc.c
- lib0.c

In [25]:
import onnx
import tvm
from tvm import relay

# 載入 ONNX 模型
onnx_model = onnx.load("./modified_model.onnx")

# 定義輸入資訊
input_name = 'float_input'
shape_dict = {input_name: (1, 4)}

# 將 ONNX 模型轉換為 Relay 模組
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設定編譯目標為產生 C 原始碼
target = tvm.target.Target("c")

# 使用 AOT 執行器
executor = tvm.relay.build_module.Executor("aot")

# 編譯 Relay 模組
with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
    lib = relay.build(mod, target=target, executor=executor, params=params)

lib.export_library("c_model.tar")

[15:18:21] /home/jovyan/project/ONNX-MLIR/tvm/src/te/schedule/bound.cc:119: not in feed graph consumer = compute(p0_red_temp, body=[T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis=[T.iter_var(k1, T.Range(0, 3), "CommReduce", "")], condition=T.bool(True), value_index=0), T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis

In [26]:
!tar xvf c_model.tar

lib1.c
devc.c
lib0.c


### 4.2 TVM 進行編譯產生
產生了兩個檔案，但不知怎利用。

- devc.c
- lib0.c

In [19]:
import tvm
from tvm import relay
from tvm.contrib import cc, utils, graph_executor
import onnx
import numpy as np

# 載入 ONNX 模型
onnx_model = onnx.load("./modified_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 確認模型的輸入名稱
shape_dict = {input_name: (1, 4)}  # 定義輸入形狀 (1, 4)
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設定編譯目標為 "c" 以生成 C 程式碼
target = "c"
with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
    lib = relay.build(mod, target=target, params=params)

# # 儲存編譯的共享庫 (dylib 為 macOS 共享庫)
# lib.export_library("logistic_regression_iris.so", cc.create_shared, cc="g++")
lib.export_library("c_model.tar")
print(f"共享庫已儲存")


共享庫已儲存


[15:08:57] /home/jovyan/project/ONNX-MLIR/tvm/src/te/schedule/bound.cc:119: not in feed graph consumer = compute(p0_red_temp, body=[T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis=[T.iter_var(k1, T.Range(0, 3), "CommReduce", "")], condition=T.bool(True), value_index=0), T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis

In [21]:
!tar xvf c_model.tar

devc.c
lib0.c
